## Demo: Simulation of QAOA on large transverse field Ising chains via free fermions

Here we show how to use the functionalities contained in tfi_ff.py
They are based on the mapping of the TFI to free fermions as discussed in [our paper](https://arxiv.org/abs/2004.14666). This solution is not original work done by us but the implementation is. Selected prior references for the mapping - using slightly different notation/bases - are 
* [Wang, Hadfield, Jiang, Rieffel - 2017](https://arxiv.org/abs/1706.02998) - also published in PRA
* [ E. Lieb, T. Schultz, and D. Mattis, Annals of Physics16,407  (1961)](https://www.math.ucdavis.edu/~bxn/lieb1961.pdf) - original work, not as close to the presentation here

In [1]:
import numpy as np
import tfi_ff

## Energy evaluation 
For applications that require only the energy expectation value for given parameters, use the eval_energy method:
Choose a number of qubits N and a number of QAOA blocks p, where the ansatz  
\begin{equation}|\psi(\vec{\theta},\vec{\varphi})\rangle = \prod_{j=p}^{1} L_X(\varphi_j)L_{ZZ}(\theta_j)|\bar{\psi}\rangle\end{equation}
will contain the ground state of the TFI model  
\begin{equation}H_{TFI} = -\sum_{k=1}^{N} Z^{(k)}Z^{(k+1)}-t\sum_{k=1}^{N}X^{(k)}\end{equation}
for $p\geq \left\lfloor N/2\right\rfloor$

In [2]:
# Fix number of qubits
N = 20
# Fix number of QAOA blocks. For $p\geq N/2$
p = 10
# Choose a transverse field strength 
t = 1.2
# Generate 2 random parameters per block (theta and phi). Periodicity of the ansatz in all parameters is pi,
# here we choose to center the parameters around 0 but with i.i.d. distribution over the whole hypercube 
# [-pi/2, pi/2]^(2p)
param = np.random.random(2*p)*np.pi-np.pi/2
# Pass the parameters, the system size and the transverse field to the evaluation function
E = tfi_ff.eval_energy(param, N, t)
print(f'The energy of the state prepared with the angles\n{np.round(param,3)}\n'
      f'has the energy per site E={E:.4f} in the TFI on {N} spins with transverse field {t}')

The energy of the state prepared with the angles
[-0.597 -0.031 -0.209  0.516  0.694  0.068 -0.058 -0.57   0.174  1.396
  0.446  1.3    1.176  0.402 -0.66  -0.142  1.101  0.537  0.021  0.792]
has the energy per site E=0.2027 in the TFI on 20 spins with transverse field 1.2


Note that our QAOA ansatz always contains an even number of parameters by design. 
In case an odd number of parameters is given, the last parameter is discarded.

One can skip the arguments $N$ and $t$ which are set to len(param) and 1 by default

## Gradient and Fubini-Study matrix evaluation
The gradient of the TFI energy can be computed via
* the standard finite difference method
* [the parameter shift rule](https://arxiv.org/abs/1811.11184)
* matrix elements of the Hamiltonian between the QAOA state and the corresponding derivative state (see below)
* [an ancilla qubit expectation value](https://arxiv.org/abs/1804.03023)

Our code makes use of the third variant, which is feasible only because all computations decompose into two-dimensional matrix-vector operations and only a small bit of memory is needed to buffer several states. In particular the combination with the Fubini-Study matrix computation, which requires the derivative states as well, makes this a practical and fast choice.  
The derivative of the energy expectation value - where we look at $\theta_j$ exemplarily -
\begin{equation}E(\vec{\theta},\vec{\varphi})=\langle\psi(\vec{\theta},\vec{\varphi})|H_{TFI}|\psi(\vec{\theta},\vec{\varphi})\rangle\end{equation}
w.r.t $\theta_j$ is given by
\begin{equation}\partial_{\theta_j}E(\vec{\theta},\vec{\varphi})=2\mathfrak{Re}\langle\psi(\vec{\theta},\vec{\varphi})|H|\partial_{\theta_j}\psi(\vec{\theta},\vec{\varphi})\rangle\end{equation}
which in the free fermion picture with $r=\lfloor N/2\rfloor$ fermions decomposes into 
\begin{align}
\partial_{\theta_j}E(\vec{\theta},\vec{\varphi})&=\sum_{q=1}^r 2\mathfrak{Re}
\langle \psi^{(q)}| H^{(q)} |\partial_{\theta_j}\psi^{(q)}\rangle \\
|\partial_{\theta_j}\psi^{(q)}\rangle &= \left(\prod_{k=N}^j R_{Z}^{(q)}(\varphi_k)R_{ax}^{(q)}(\theta_k)\right)W^{(q)}\left(\prod_{k=j-1}^1 R_{Z}^{(q)}(\varphi_k)R_{ax}^{(q)}(\theta_k)\right)|\bar{\psi}\rangle
\end{align}

with $W^{(q)}=\cos \alpha_q Z + \sin\alpha_q Y$ and some fermion-specific angles $\alpha_q$. This means that while evolving the $q$-th fermion state with the QAOA unitaries, we can split off a copy at the $2j$-th unitary in order to apply the respective $W^{(q)}$ (for $\partial_{\theta_j}$) or $Z$ (for $\partial_{\varphi_j}$) operator to it and continue evolving this state with the remaining QAOA unitaries in parallel to the original, derivative-free state.

Evidently, we need these states to compute the Fubini-Study matrix ${}^{[1]}$
\begin{equation}
F_{ij} = \mathfrak{Re}\left\{\langle\partial_i\psi|\partial_j\psi\rangle\right\}-\langle\partial_i\psi|\psi\rangle\langle\psi|\partial_j\psi\rangle
\end{equation}
such that the gradient uses resources that are required by $F$ in any case but computes some matrix elements of $H$ with them rather than the overlaps in F, which are independent of the Hamiltonian.



[1] For convenience, relabel $\partial_j=\partial_{\theta_{j/2}}$ for even $j$ and $\partial_j=\partial_{\varphi_{(j-1)/2}}$ for odd $j$


In [3]:
# As before specify number of qubits N, number of blocks p and transverse field
N = 10; p=5; t=1.2;
# Generate random parameters as before 
param = np.random.random(2*p)*np.pi-np.pi/2
# The entire computation is obscured in one method here because the computation of the gradient and F 
# are more efficient when performed together.
E, grad, F = tfi_ff.eval_all(param, N, t)
print(f'The energy of the state prepared with the angles\n{np.round(param,3)}\n'
      f'has the energy per site E={E:.4f} in the TFI on {N} spins with transverse field {t}.\n\n'
      f'The gradient is {np.round(grad,3)}\n'
      f'and the Fubini-Study matrix reads\n{np.round(F,3)}')

The energy of the state prepared with the angles
[ 0.794  0.07   1.105  0.759  0.767  0.054  0.701 -1.394  1.263  0.976]
has the energy per site E=-0.2743 in the TFI on 10 spins with transverse field 1.2.

The gradient is [-0.065 -0.049 -0.123  0.987  0.026  0.448  0.056  0.372  1.302  0.026]
and the Fubini-Study matrix reads
[[ 2.5   -0.     2.482  0.07   1.408  1.689  1.368  1.016 -1.463  0.288]
 [-0.     3.145 -0.083 -0.388  2.256  0.095  2.184  0.853  0.767 -0.834]
 [ 2.482 -0.083  2.476 -0.091  1.353  1.602  1.305  1.005 -1.502  0.344]
 [ 0.07  -0.388 -0.091  2.919 -0.558  1.325 -0.38  -0.357  0.352 -0.445]
 [ 1.408  2.256  1.353 -0.558  3.171 -0.022  3.099  0.764 -0.222 -0.807]
 [ 1.689  0.095  1.602  1.325 -0.022  3.313 -0.057  2.204 -0.819  0.833]
 [ 1.368  2.184  1.305 -0.38   3.099 -0.057  3.046  0.572 -0.189 -0.881]
 [ 1.016  0.853  1.005 -0.357  0.764  2.204  0.572  3.295 -0.492  0.99 ]
 [-1.463  0.767 -1.502  0.352 -0.222 -0.819 -0.189 -0.492  4.151  0.462]
 [ 0.288 -0.834

In [4]:
# We restricted ourselves to 10 qubits here for readability of the output, but regarding the computation time, 
# there are not strong constraints as the algorithm scales polynomially in the system size and the number of 
# parameters, with very very small prefactors:
import time

# Choose some system sizes and fix p=N/2
for N in 10*np.arange(1,7):
    start = time.process_time()
    E, g, F = tfi_ff.eval_all(np.random.random(N)*np.pi-np.pi/2)
    end = time.process_time()
    print(f'For N={N} qubits and p={N//2:2} QAOA blocks the computation took {end-start:.2f} seconds.')

For N=10 qubits and p= 5 QAOA blocks the computation took 0.01 seconds.
For N=20 qubits and p=10 QAOA blocks the computation took 0.07 seconds.
For N=30 qubits and p=15 QAOA blocks the computation took 0.16 seconds.
For N=40 qubits and p=20 QAOA blocks the computation took 0.43 seconds.
For N=50 qubits and p=25 QAOA blocks the computation took 1.01 seconds.
For N=60 qubits and p=30 QAOA blocks the computation took 1.78 seconds.


### Possible improvements
The code is fast because the problem is easy. There will be room for improvement for the unlikely case that a faster performance is required. Some ideas for improvement are:
* pre-compute the QAOA blocks as derivative generators W^q can be pulled out of these blocks such that all time evolutions only require the blocks and not the separate layers.
* the blocks in the previous point could additionally be precomputed analytically, saving many of the matrix multiplications.
* port the code to C/C++, which should be easily doable as it only contains matrix-vector and vector-vector multiplications.
